In [1]:
import torch
from GCN.data_processing import Data_Loader
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
import csv
import numpy as np
import io
import os, random
import time
import json
import torch
import zipfile
import numpy as np
import torch.nn as nn
from PIL import Image,ImageOps
import torch.nn.functional as F
from einops import rearrange, repeat
import math
from torch import einsum
from argparse import ArgumentParser
from core.models.curvenet_cls import CurveNet
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import argparse
random_seed = 420 #for reproducibility
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torchsummary import summary
import colorama
from colorama import Fore, Back, Style
colorama.init(autoreset=True)
np.seterr(invalid='ignore')

/home/shafin/Desktop/Rehabilitation-Rafat/cvRehab/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
!nvidia-smi

Mon Feb 20 11:28:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 35%   38C    P0   125W / 350W |    194MiB / 24267MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
# print(torch.cuda.current_device())
# print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 3090


In [4]:
import numpy as np
from IPython.core.debugger import set_trace

class Graph():
    def __init__(self, num_node):
        self.num_node = num_node
        self.AD, self.AD2, self.bias_mat_1, self.bias_mat_2 = self.normalize_adjacency()
        
    def normalize_adjacency(self):
        self_link = [(i, i) for i in range(self.num_node)]
#         print('self', self_link)
        neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                                      (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                                      (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                                      (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                                      (22, 23), (23, 8), (24, 25), (25, 12)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
#         print('neighbour: ', neighbor_link)
        edge = self_link + neighbor_link
#         print('Edge: ' ,edge)
        A = np.zeros((self.num_node, self.num_node)) # adjacency matrix
        print(A.shape)
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        print(A)
        A2 = np.zeros((self.num_node, self.num_node)) # second order adjacency matrix
        for root in range(A.shape[1]):
            for neighbour in range(A.shape[0]):
                if A[root, neighbour] == 1:
                    for neighbour_of_neigbour in range(A.shape[0]):
                        if A[neighbour, neighbour_of_neigbour] == 1:
                            A2[root,neighbour_of_neigbour] = 1  
#         print(A2)
        #AD = self.normalize(A)
        #AD2 = self.normalize(A2)
        bias_mat_1 = np.zeros(A.shape)
        bias_mat_2 = np.zeros(A2.shape)
        bias_mat_1 = np.where(A!=0, bias_mat_1, -1e9)
        bias_mat_2 = np.where(A2!=0, A2, -1e9)
        AD = A.astype('float32')
        AD2 = A2.astype('float32')
        bias_mat_1 = bias_mat_1.astype('float32')
        bias_mat_2 = bias_mat_2.astype('float32')
        AD = torch.tensor(AD)
        AD2= torch.tensor(AD2)
        bias_mat_1 = torch.tensor(bias_mat_1)
        bias_mat_2 = torch.tensor(bias_mat_2)
        return AD, AD2, bias_mat_1, bias_mat_2
        
    def normalize(self, adjacency):
        rowsum = np.array(adjacency.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0
        r_mat_inv = np.diag(r_inv)
        normalize_adj = r_mat_inv.dot(adjacency)
        normalize_adj = normalize_adj.astype('float32')
        normalize_adj = torch.tensor(normalize_adj)   
        return normalize_adj

In [5]:
# print(len(np.unique(train_y)))
# print(np.unique(train_y))

In [6]:
# print(len(train_x))
# print(len(train_y))
# print(len(test_x))
# print(len(test_y))

In [7]:
# train_x.shape

In [8]:
# train_y[0].shape

In [9]:
# x = torch.tensor(train_x)
# x.size()

In [10]:
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()

    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d

        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']

        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)

        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d

        self.embed_fns = embed_fns
        self.out_dim = out_dim

    def embed(self, inputs):
        normalized = torch.cat([fn(inputs) for fn in self.embed_fns], -1)
        return normalized
    
def get_embedder(multires = 10, i=0):
    if i == -1:
        return nn.Identity(), 1

    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 1,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }

    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

embeder = get_embedder()[0]    


In [11]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


# GELU -> Gaussian Error Linear Units
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


class RemixerBlock(nn.Module):
    def __init__(
        self,
        dim,
        seq_len,
        causal = False,
        bias = False
    ):
        super().__init__()
        self.causal = causal
        self.proj_in = nn.Linear(dim, 2 * dim, bias = bias)
        self.mixer = nn.Parameter(torch.randn(seq_len, seq_len))
        self.alpha = nn.Parameter(torch.tensor(0.))
        self.proj_out = nn.Linear(dim, dim, bias = bias)

    def forward(self, x):
        mixer, causal, device = self.mixer, self.causal, x.device
        x, gate = self.proj_in(x).chunk(2, dim = -1)
        x = F.gelu(gate) * x

        if self.causal:
            seq = x.shape[1]
            mask_value = -torch.finfo(x.dtype).max
            mask = torch.ones((seq, seq), device = device, dtype=torch.bool).triu(1)
            mixer = mixer[:seq, :seq]
            mixer = mixer.masked_fill(mask, mask_value)

        mixer = mixer.softmax(dim = -1)
        mixed = einsum('b n d, m n -> b m d', x, mixer)

        alpha = self.alpha.sigmoid()
        out = (x * mixed) * alpha + (x - mixed) * (1 - alpha)

        return self.proj_out(out)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        """
        Args:
            x: `embeddings`, shape (batch, max_len, d_model)
        Returns:
            `encoder input`, shape (batch, max_len, d_model)
        """
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)


class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        
        # print(f'Attention:: {dim} - {heads} - {dim_head} - {dropout}')

        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.pos_embedding = PositionalEncoding(dim,0.1,240)
        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x += self.pos_embedding(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()

        # print('\n')
        # print(f'Transformers:: {dim} - {depth} - {heads} - {dim_head} - {mlp_dim}')

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
                #PreNorm(dim, RemixerBlock(dim,17))
            ]))

    def forward(self, x, swap = False):
        if swap: # for the self.transformer(x,swap = True)
            b, t, n , c = x.size() 
        for idx, (attn, ff) in enumerate(self.layers):
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    #* attention along with all timesteps(frames) for each point(landmark)
                    x = rearrange(x, "b t n c -> (b n) t c")
                else:
                    #* attention to all points(landmarks) in each timestep(frame)
                    x = rearrange(x, "b t n c -> (b t) n c")
            x = attn(x) + x  # skip connections
            x = ff(x) + x    # skip connections
            
            # Now return the input x to its original formation
            if swap: # for the self.transformer(x,swap = True)
                if idx % 2 == 0:
                    x = rearrange(x, "(b n) t c -> b t n c", b = b)
                else:
                    x = rearrange(x, "(b t) n c -> b t n c", b = b)
                
        return x


class TemporalModel(nn.Module):
    
    def __init__(self):
        super(TemporalModel,self).__init__()
                
        self.encoder  =  CurveNet() # curve aggregation, needed for Point Clouds Shape Analysis. 
        self.downsample = nn.Sequential(
                            nn.Conv1d(117, 32, kernel_size=1, bias=False),
                            nn.BatchNorm1d(32),
                            # nn.Dropout(p=0.25), #* NEW
                            #nn.ReLU(inplace=True),
                            #nn.Conv1d(128, 32, kernel_size=1, bias=False),
                            #nn.BatchNorm1d(32),
                            )
        
        self.transformer = Transformer(256, 6, 4, 256//4, 256 * 2, 0.1)
        self.time = Transformer(256, 3, 4, 256//4, 256 * 2, 0.1)
        self.dropout = nn.Dropout(0.1)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(256),
            nn.Linear(256,1),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        b,t,n,c = x.size()
    
        x = rearrange(x, "b t n c -> (b t) c n")
        x = rearrange(self.dropout(self.encoder(x)), "b c n -> b n c") 
        x = self.downsample(x).view(b,t,32,-1) #b t 32 c
        x = self.transformer(x,swap = True).view(b,t,-1,256).mean(2)
        x = self.time(x).mean(1)
        x = self.mlp_head(x)
        return x

min_xyz = np.array([0.06372425, 0.05751023, -0.08976112]).reshape(1,1,3)
max_xyz = np.array([0.63246971, 1.01475966, 0.14436169]).reshape(1,1,3)


In [12]:
def load_data(path, nr = 80, n_dim= 117, timesteps= 240):
    f = open('./UI-PRMD/'+path+'/Data_Correct.csv')
    csv_f = csv.reader(f)
    Correct_X = list(csv_f)

    # Convert the input sequences into numpy arrays
    train_input1 = np.asarray(Correct_X, dtype = float)
    correct_input = np.zeros((nr,timesteps,n_dim))
    for i in range(len(train_input1)//n_dim):
           correct_input[i,:,:] = np.transpose(train_input1[n_dim*i:n_dim*(i+1),:])
    
    f = open('./UI-PRMD/'+path+'/Labels_Correct.csv')
    csv_f = csv.reader(f)
    Correct_Y = list(csv_f)
    
    # Convert the input labels into numpy arrays
    correct_label = np.asarray(Correct_Y, dtype = float)
    
    f = open('./UI-PRMD/'+path+'/Data_Incorrect.csv')    
    csv_f = csv.reader(f)
    Incorrect_X = list(csv_f)

    # Convert the input sequences into numpy arrays
    test_input1 = np.asarray(Incorrect_X)
    n_dim = 117
    incorrect_input = np.zeros((nr,timesteps,n_dim))
    for i in range(len(test_input1)//n_dim):
          incorrect_input[i,:,:] = np.transpose(test_input1[n_dim*i:n_dim*(i+1),:])
            
    f = open('./UI-PRMD/'+path+'/Labels_Incorrect.csv')
    csv_f = csv.reader(f)
    Incorrect_Y = list(csv_f)
    
    # Convert the input labels into numpy arrays
    incorrect_label = np.asarray(Incorrect_Y, dtype = float)
    
    return correct_input, correct_label, incorrect_input, incorrect_label

In [13]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def performance_metrics(test_y, y_pred):
    test_dev = abs(test_y-y_pred)
#     mean_abs_dev = torch.mean(test_dev)
    mae = mean_absolute_error(test_y, y_pred)
    rms_dev = sqrt(mean_squared_error(y_pred, test_y))
    mse = mean_squared_error(test_y,y_pred) 
    mape = mean_absolute_percentage_error(test_y, y_pred)
    return mae, rms_dev, mse, mape

In [14]:
def train(epochs,training_generator,test_generator,file, data_loader = None):
    
    con = []      
    net = TemporalModel()
    net.cuda()
    eval_loss = 500
    lr = 0.0001
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr,weight_decay= 0.0)
#     lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000], gamma=0.1)
    loss_func = torch.nn.HuberLoss(reduction='mean', delta= 0.1)
    start_time = time.time()
    best_accuracy = 500

    for epoch in range(epochs):
        pred_label = []
        true_label = []
        number_batch = 0
        for x, y in tqdm(training_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):
            if torch.cuda.device_count() > 0:
                x = x.cuda()
                y = y.cuda()
            x = torch.unsqueeze(x, 1) 
#             print(x.size())
            b,d,t,n,c = x.size()
            x = x.view(-1,t,n,c)
            pred = net(x)
            loss = loss_func(pred,y)
            
            
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            
            pred_label.extend(pred.flatten())
            true_label.extend(y.flatten())
            number_batch += 1
#             lr = lr_scheduler.get_last_lr()[0]

#         lr_scheduler.step()
        
#         print(true_label)
#         print(pred_label)
#         pred_label = torch.cat(pred_label,0)
#         true_label = torch.cat(true_label,0)
#         train_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
        pred_label = torch.tensor(pred_label)
        true_label = torch.tensor(true_label)
        mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
    
        output('Epoch: ' + 'train ' + str(epoch) + 
              ', mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
                ', mape: ' + str(mape)
              )
        print('Epoch: ' + 'train ' + str(epoch) + 
              ', mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape)
              )
        net.eval()
        
        pred_label = []
        pred_avg   = []
        true_label = []
        with torch.no_grad():
          for x, y in tqdm(test_generator, desc=f"Epoch {epoch}/{epochs-1}", ncols=60):

              if torch.cuda.device_count() > 0:
                  x = x.cuda()
                  y = y.cuda()
              x = torch.unsqueeze(x, 1)
              b,d,t,n,c = x.size()
              x = x.view(-1,t,n,c)
              pred_y    = net(x)
            ###commented to convert round
#               pred_mean = (pred_y.view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
#               pred_y    = ((pred_y).view(b,d).mean(1,keepdim = True) >= 0.5).float().cpu().detach()
              
              pred_label.extend(pred_y)
              true_label.extend(y)
# #           pred_avg   = torch.cat(pred_avg,0)  
# #           true_label = torch.cat(true_label,0)
          pred_label = torch.tensor(pred_label)
          true_label = torch.tensor(true_label)
          loss_e = loss_func(pred_label,true_label)
          pred_label = torch.unsqueeze(pred_label,1)
          true_label = torch.unsqueeze(true_label,1)
          
          if data_loader is not None:
              pred_label = data_loader.sc2.inverse_transform(pred_label)
              true_label = data_loader.sc2.inverse_transform(true_label)
                
#           print(pred_label)
#           print(true_label)
          mae, rms_dev, mse, mape = performance_metrics(true_label, pred_label)
#           test_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
#           test_avg      = torch.sum(pred_avg   == true_label).type(torch.FloatTensor) / true_label.size(0)
          con.append([epoch,mae])
          output('test accuracy: ' + 
              'mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape) +
                 ', loss' + str(loss_e)
              )
          print(Fore.GREEN + 'test accuracy: ' + 
              'mae ' + str(mae) + 
              ', rms_dev: ' + str(rms_dev) +
              ', mse: ' + str(mse) +
              ', mape: ' + str(mape)+
                ', loss' + str(loss_e)
              )
          print()
          if mae < best_accuracy:
              filepath = f"{file}-{epoch:}-{loss}-{mae}.pt"
              torch.save(net.state_dict(), filepath)
            #   torch.save(net, filepath)
            #   test_frames(f'{test_accuracy}={test_f}')
              output('Best Results Achieved!!!!!!!!!!!!')
              print("Best Results Achieved!!!!!!!!!!!!")
              best_accuracy = mae

        net.train()
        
        output(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")
        # print(f"ETA Per Epoch:{(time.time() - start_time) / (epoch + 1)}")

    print("Lowest loss: ", eval_loss)
    
    
# image_size = 48
# label_path = "labels"
# data = "npy"

# sometimes = lambda aug: va.Sometimes(0.5, aug)
# seq = va.Sequential([
#     va.RandomCrop(size=(image_size, image_size)),       
#     sometimes(va.HorizontalFlip()),              
# ])


# label_path = "labels"


In [15]:
from sklearn.preprocessing import MinMaxScaler
class prmd_scaler():
    def __init__(self, y_train):
        self.y_train = y_train
        self.sc2 = MinMaxScaler()
        self.scaled_y = self.preprocessing()
    def preprocessing(self):
        y = self.sc2.fit_transform(self.y_train)
        return None

In [16]:
def main():
    
    # Add the arguments
    ####################Parser not working###########################
    ex = "UI_PRMD_ex3"
    lr = 0.0001
    epoch = 2000
    batch_size = 5
    
    ####################Parser not working###########################

    # my_parser.add_argument('--ex', type=str, default="Kimore_ex5",
    #                        help='the name of exercise.', required=True)
    # my_parser.add_argument('--lr', type=int, default= 0.0001,
    #                        help='initial learning rate for optimizer.')
    # my_parser.add_argument('--epoch', type=int, default= 1000,
    #                        help='number of epochs to train.')
    # my_parser.add_argument('--batch_size', type=int, default= 10,
    #                        help='training batch size.')
    # args = my_parser.parse_args()
    global output
    def output(s):
        with open(f"log_{ex}","a") as f:
            f.write(str(s) + "\n")
            
    if "Kimore" in ex:
        data_loader = Data_Loader(ex) 
        train_x, test_x, train_y, test_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, 
                                                            random_state = random_seed)
        
    else:
        nr = 80
        Correct_data, Correct_label, Incorrect_data, Incorrect_label = load_data(ex)
        print(Correct_data.shape[0])
        print(Correct_label.shape)
        trainidx1 = random.sample(range(0,Correct_data.shape[0]),int(nr*0.8))
        trainidx2 = random.sample(range(0,Incorrect_data.shape[0]),int(nr*0.8))
        valididx1 = np.setdiff1d(np.arange(0,nr,1),trainidx1)
        valididx2 = np.setdiff1d(np.arange(0,nr,1),trainidx2)
        # Training set: data and labels
        train_x = np.concatenate((Correct_data[trainidx1,:,:],Incorrect_data[trainidx2,:,:]))
        train_y = np.concatenate((np.squeeze(Correct_label[trainidx1]),np.squeeze(Incorrect_label[trainidx2])))

        # Validation set: data and labels
        test_x = np.concatenate((Correct_data[valididx1,:,:],Incorrect_data[valididx2,:,:]))
#         print(valid_x.shape, 'validation data')
        test_y = np.concatenate((np.squeeze(Correct_label[valididx1]),np.squeeze(Incorrect_label[valididx2])))
        
#         print(valid_y.shape, 'validation labels')
        
    
    
    print(train_x.shape)
    print(train_y.shape)
    print(test_x.shape)
    print(test_y.shape)
    train_x = torch.Tensor(train_x)
    train_y = torch.Tensor(train_y)
    test_x  = torch.Tensor(test_x)
    test_y = torch.Tensor(test_y)
    if "UI" in ex:
        train_x = torch.unsqueeze(train_x, 3)
        train_y = torch.unsqueeze(train_y, 1)
        train_x = train_x.expand(train_x.size(dim = 0),train_x.size(dim = 1),train_x.size(dim = 2),3)
        print(train_x.size())
        test_x = torch.unsqueeze(test_x,3)
        test_y = torch.unsqueeze(test_y, 1)
        test_x= test_x.expand(test_x.size(dim = 0), test_x.size(dim = 1), test_x.size(dim = 2),3)
        print(test_x.size())
#         data_loader = prmd_scaler(train_y)
#         train_y = data_loader.scaled_y
# #         test_y = data_loader.scaled_yt
# #         print(test_y[:10])
#         train_y = torch.Tensor(train_y)
#         test_y = torch.Tensor(test_y)
        print(train_y.size())
        print(test_y.size())
        
        
    dataset = torch.utils.data.TensorDataset(train_x, train_y)
    training_generator = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True)
    dataset = torch.utils.data.TensorDataset(test_x, test_y)
    test_generator = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = False)
    current_path = './models_UI/'
    train(epoch,training_generator,test_generator,current_path)


In [17]:
if __name__ == "__main__":
    main()

80
(80, 1)
(128, 240, 117)
(128,)
(32, 240, 117)
(32,)
torch.Size([128, 240, 117, 3])
torch.Size([32, 240, 117, 3])
torch.Size([128, 1])
torch.Size([32, 1])


Epoch 0/1999: 100%|█████████| 26/26 [00:13<00:00,  1.96it/s]


Epoch: train 0, mae 0.04365321, rms_dev: 0.08236863091101498, mse: 0.0067845914, mape: 4.807431995868683


Epoch 0/1999: 100%|███████████| 7/7 [00:00<00:00,  7.27it/s]


test accuracy: mae 0.021331478, rms_dev: 0.02392705845829802, mse: 0.0005725041, mape: 2.3231156170368195, losstensor(0.0003)

Best Results Achieved!!!!!!!!!!!!


...

test accuracy: mae 0.0071451366, rms_dev: 0.010785796637064615, mse: 0.00011633341, mape: 0.7856221869587898, losstensor(5.8167e-05)



Epoch 1999/1999: 100%|██████| 26/26 [00:12<00:00,  2.13it/s]


Epoch: train 1999, mae 0.00066985795, rms_dev: 0.0008818942424589804, mse: 7.7773745e-07, mape: 0.07338140276260674


Epoch 1999/1999: 100%|████████| 7/7 [00:01<00:00,  6.97it/s]


test accuracy: mae 0.007021038, rms_dev: 0.01058060493545404, mse: 0.0001119492, mape: 0.7733155973255634, losstensor(5.5975e-05)

Lowest loss:  500


In [18]:

# Print the size of the data 
print(Correct_data.shape, 'correct sequences')
print(Correct_label.shape, 'correct labels')
print(Incorrect_data.shape, 'incorrect sequences')
print(Incorrect_label.shape, 'incorrect labels')

NameError: name 'Correct_data' is not defined